In [ ]:
%pip install transformers langdetect deep-translator torch torchvision torchaudio langchain langchain-community langchain-ollama gradio -q

In [ ]:
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate

In [ ]:
model_name = "mistral"
llm = ChatOllama(
    temperature = 0,
    max_tokens = 2000,
    model = model_name
)

template = """
You're a native internal AI assistant. Help users with their important tasks, like a professor in a particular field.

Query: {query}
"""

prompt = PromptTemplate(
    input_variables = ["query"],
    template = template
)

In [ ]:
chain = prompt | llm

In [ ]:
from langdetect import detect
from deep_translator import GoogleTranslator

class Translation:
  def __init__(self, text, destination):
    self.text = text
    self.destination = destination

    try:
      self.original = detect(self.text)
    except Exception as error:
      print(f"Error Occurred, Details : {error}")
      self.original = "auto"

  def translate(self):
    translator = GoogleTranslator(source=self.original, target=self.destination)
    translation = translator.translate(self.text)

    return translation

In [ ]:
import gradio as gr

In [ ]:
def reply(message, history):
  txt = Translation(message, "en")

  if txt.original == "en":
    response = chain.invoke({
        "query": message
    })

    return response
  else:
    translation = txt.translate()

    response = chain.invoke({
        "query": translation
    })

    t = Translation(response.content, txt.original)
    final_response = t.translate()

    return final_response

In [ ]:
question = "What is Deep Learning?"

result = reply(question, history = [])

In [ ]:
result.content

In [ ]:
question = "ரியல் எஸ்டேட்டில் அதிக பணம் சம்பாதிப்பது எப்படி?"

result = reply(question, history = [])

result

In [ ]:
demo = gr.ChatInterface(fn=reply, title = "Multi-Lingual ChatBot")

demo.launch(debug=False, share=False)